In [ ]:
import os
import spacy
import random
import speech_recognition as sr
import mysql.connector

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

try:
    # Establish a connection to the MySQL database
    conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='workshopmanagement'
    )
    cursor = conn.cursor()
    print("Connected to the database.")
except mysql.connector.Error as e:
    print(f"Error connecting to the database: {e}")
    exit()

def listen_to_user():
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        print("Listening to user... Speak now.")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source, timeout=5)

    try:
        print("Recognizing...")
        user_input = recognizer.recognize_google(audio)
        print("User said:", user_input)
        return user_input
    except sr.UnknownValueError:
        print("Sorry, could not understand audio. Please try again.")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

def get_user_input():
    print("Welcome to the JABAL SARHAD AUTOMOTIVE DIAGNOSTIC ASSISTANT !")

    while True:
        print("Select the type of work needed:")
        print("1. Mechanical")
        print("2. Electrical")
        print("3. AC")

        work_type = input("Enter the corresponding number (1, 2, or 3): ")

        if work_type in ["1", "2", "3"]:
            break
        else:
            print("Invalid choice. Please choose 1, 2, or 3.")

    voice_input = input("Would you like to describe the issue verbally? (yes/no): ").lower()

    if voice_input == "yes":
        issue_description = listen_to_user()
        if issue_description:
            print("Description received using voice recognition:", issue_description)
        else:
            print("Failed to recognize audio. Please provide a description manually.")
    else:
        issue_description = input("Describe the issue with your car: ")

    return int(work_type), issue_description

def diagnose_and_solve_issue(work_type, issue_description):
    if work_type == 1:
        return diagnose_mechanical_issue(issue_description)
    elif work_type == 2:
        return diagnose_electrical_issue(issue_description)
    elif work_type == 3:
        return diagnose_ac_issue(issue_description)
    else:
        return "Invalid choice. Please choose 1, 2, or 3."

def diagnose_mechanical_issue(issue_description):
    # Custom logic for mechanical issue diagnosis using spaCy
    # Here, we can define rules or patterns based on common phrases
    doc = nlp(issue_description.lower())

    # Fetch issues and solutions from the database
    cursor.execute("SELECT IssueDescription, Solution FROM mechanicalissues")
    issues_mapping = {issue[0].lower(): issue[1] for issue in cursor.fetchall()}

    for token in doc:
        for issue_description, solution in issues_mapping.items():
            if token.text in issue_description:
                return solution

    return "Issue not recognized. Please provide more details."

def diagnose_electrical_issue(issue_description):
    # Custom logic for electrical issue diagnosis using spaCy
    # Here, we can define rules or patterns based on common phrases
    doc = nlp(issue_description.lower())

    # Fetch issues and solutions from the database
    cursor.execute("SELECT IssueDescription, Solution FROM electricalissues")
    issues_mapping = {issue[0].lower(): issue[1] for issue in cursor.fetchall()}

    for token in doc:
        for issue_description, solution in issues_mapping.items():
            if token.text in issue_description:
                return solution

    return "Issue not recognized. Please provide more details."

def diagnose_ac_issue(issue_description):
    # Custom logic for AC issue diagnosis using spaCy
    # Here, we can define rules or patterns based on common phrases
    doc = nlp(issue_description.lower())

    # Fetch issues and solutions from the database
    cursor.execute("SELECT IssueDescription, Solution FROM acissues")
    issues_mapping = {issue[0].lower(): issue[1] for issue in cursor.fetchall()}

    for token in doc:
        for issue_description, solution in issues_mapping.items():
            if token.text in issue_description:
                return solution

    return "Issue not recognized. Please provide more details."

# Get user input using the modified function
work_type, issue_description = get_user_input()

# Ensure cursor is defined before proceeding
if 'cursor' not in locals() or cursor is None:
    print("Error: Database connection issue.")
    exit()

# Diagnose the issue and provide a solution
solution = diagnose_and_solve_issue(work_type, issue_description)

# Close the database connection
conn.close()

# Display the result
print("\nDiagnosis Result:")
print(solution)


Connected to the database.
Welcome to the JABAL SARHAD AUTOMOTIVE DIAGNOSTIC ASSISTANT !
Select the type of work needed:
1. Mechanical
2. Electrical
3. AC
